## Initialize environment

Note: Before executing this notebook on colab, copy your .env file in /content/drive/MyDrive/.env or otherwise you'll be prompted to enter your GEMINI_API_KEY all the times.

In [ ]:
! uv pip install agentics-py
! uv pip install 'crewai[tools]'

import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive
    drive.mount("/content/drive")
    load_dotenv("/content/drive/MyDrive/.env")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Using Python 3.12.11 environment at: /Users/gliozzo/miniforge3/envs/agentics
Audited 1 package in 14ms
Using Python 3.12.11 environment at: /Users/gliozzo/miniforge3/envs/agentics
Audited 1 package in 9ms
In Colab: False


## Create a new tool
Wrap the Duck Duck Go Search API call into a CrewAI tool using the @tool decorator. Similarly you can turn any function of your choice into a tool. 

In [2]:
from ddgs import DDGS
from crewai.tools import tool
## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("fetch_data_async")
def web_search(query: str) -> str:
    """ Fetch web search results for the given query using DDGS."""
    return str(DDGS().text(query, max_results=10))    


## Define your agent, task and crew

Note that the agent is provided with memory = True This provides the agent with conversation history. By activating memory=True in the crew, we make sure that the agents persists across multiple crew session calls, with different input states. 

In [3]:
from crewai import Agent, Task, Crew, Process

# Create a conversational agent with a friendly role/goal
chat_agent = Agent(
    role="Helpful Assistant",
    goal="Have a natural multi-turn conversation",
    backstory="You are a friendly assistant that remembers context and asks for clarification when needed.",
    memory=True  # enable memory for conversational context. 
)

# Define a simple Task that represents a single AI response
task = Task(
    description="Respond appropriately to user's message, maintaining context. {input}",
    expected_output="A conversational reply",
    agent=chat_agent,
    tools=[web_search]
)

crew = Crew(
    agents=[chat_agent],
    tasks=[task],
    memory=True,  # ensure conversation state persists across turns
)

## Run the crew in a conversation loop
Since memory has been set to True, the crew will get access to the previous round of conversations, exibiting a conversational behaviour

In [5]:
while user_input := input("You: ").strip():
    print("\n\n===========\nYou: ",user_input)
    result = crew.kickoff(inputs={"input": user_input})
    print("\nAssistant:", result)



You:  What is agentics?

Assistant: IBM Agentics is a platform developed by IBM that facilitates the creation and orchestration of autonomous AI systems, referred to as agentic AI. These systems are designed to execute tasks and make decisions with minimal human supervision, thereby enhancing organizational efficiency. The primary focus of IBM Agentics is to streamline workflows, automate processes, and improve productivity across various business functions.

One of the standout features of IBM Agentics is its ability to integrate seamlessly with existing business infrastructures, such as AWS, Salesforce, and ServiceNow. This integration helps organizations leverage their current systems while adopting advanced AI capabilities without extensive overhauls.

Furthermore, IBM's approach to agentic AI emphasizes the importance of balancing creativity from generative AI with operational control. This balance ensures that organizations can deploy AI responsibly while adhering to governance

## Let's give it one more tool